In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from NanTex_backend.Util.pyDialogue import pyDialogue as pD
from NanTex_backend.data_preparation.overlay_generation import OverlayGenerator

# LINUX

In [ ]:
## Glob data
from glob import glob

root_feature_1: str = "path/to/feature_1"
root_feature_2: str = "path/to/feature_2"
root_feature_3: str = "path/to/feature_3"

root_features: list[str] = [root_feature_1, root_feature_2, root_feature_3]
root_features = [glob(f"{root}/*.npy") for root in root_features]

In [ ]:
## setup
OLGen: OverlayGenerator
OLGen = OverlayGenerator.from_glob(
    *root_features, data_path_out=None, mode="rotation", multi_core=True, DEBUG=True
)

# Windows

In [ ]:
## setup
OLGen: OverlayGenerator
OLGen = OverlayGenerator.from_explorer(
    data_path_out=None,
    mode="rotation",
    multi_core=False,
    DEBUG=True,
    dtype_out=np.uint16,
    dtype_in=np.uint8,
)

In [ ]:
from copy import deepcopy

backup = deepcopy(OLGen.data_in)

## Check Metadata

In [ ]:
OLGen.metadata

## Single Core Test

In [ ]:
OLGen.data_in = deepcopy(backup)

In [ ]:
OLGen.multi_core = False
OLGen.mode = "overlay"
OLGen.generate_overlay()

## Multi Core Test

In [ ]:
OLGen.mode = "rotation"
OLGen.multi_core = True
OLGen.metadata["sleeptime"] = 0.0
OLGen.metadata["dtype"] = np.uint16

In [ ]:
OLGen.setup_multi_core(num_cpu=12, num_gpu=0, launch_dashboard=True)

In [ ]:
OLGen.generate_overlay(patchsize=(512, 512), max_val=255)

In [ ]:
OLGen.shutdown_multi_core()

## Test Patching

In [ ]:
OLGen.patches = 16
OLGen.dtype_out = np.float32
OLGen.disable_auto_standardization = False
OLGen.multi_core = True

In [ ]:
OLGen.setup_multi_core(num_cpu=2, launch_dashboard=True)

In [ ]:
OLGen.generate_overlay()

## Test Augmented Patching

In [ ]:
OLGen.patches = 16
OLGen.imagesize = (1024 * 5, 1024 * 5)
OLGen.patch_content_ratio = 0.1
OLGen.dtype_out = np.float32
OLGen.dtype_in = np.uint8
OLGen.multi_core = False
OLGen.disable_auto_standardization = False
OLGen.augment = True
OLGen.mode = "rotation"

In [ ]:
OLGen.setup_multi_core(num_cpu=3, launch_dashboard=True)

In [ ]:
## Augmentation line
import albumentations as A

# parts
parts = [A.VerticalFlip(p=1), A.HorizontalFlip(p=1), A.RandomRotate90(p=1)]

# full
OLGen.augmentation_pipeline = A.Compose(parts)

In [ ]:
OLGen.generate_overlay()

In [ ]:
OLGen.metadata

## Check the Output

In [ ]:
## checking the setup
from glob import glob
import numpy as np
from NanTex_backend.Util.pltStyler import pltStyler

pltStyler().enforece_stylesheet()

# grab files we just wrote
files = glob(f"{OLGen.data_path_out}/*.npy")
file = np.random.choice(files)
data = np.load(file)

# talk to operator
print(f"File selected: {file}")
print(f"Data shape: {data.shape}\ndtype: {data.dtype}")
for i in range(data.shape[0]):
    print(
        f" - Channel {i + 1}: min={data[i].min():.3f}, max={data[i].max():.3f}, mean={data[i].mean():.3f}, std={data[i].std():.3f}"
    )

# plot output
fig, axs = plt.subplots(1, data.shape[0], figsize=(5 * data.shape[0], 5))
for i in range(data.shape[0]):
    axs[i].imshow(data[i], cmap="gray")
    axs[i].set_title(f"Channel {i + 1}")
plt.show()

## DEBUG

In [ ]:
OLGen.__pad_input_imgs__()

In [ ]:
OLGen.data_in

In [ ]:
## Punchcard generation
OLGen.generate_punchcards()
OLGen.data_punchcards

In [ ]:
OLGen.data_paths_in

In [ ]:
## Punchcard generation
OLGen.mode = "rotation"
OLGen.multi_core = False
OLGen.generate_punchcards()
OLGen.data_punchcards

In [ ]:
from NanTex_backend.data_preparation.overlay_generation import OVERLAY_HELPER

In [ ]:
punchcard = OLGen.data_punchcards["3_feature_rotation_(0, 0, 0, 3, 0, 2)"]
data_in = OLGen.data_in

out = [
    data_in[key][value]
    for key, value in punchcard.items()
    if key not in ["rotation", "dtype"]
]
print(out[0].shape, out[1].shape, out[2].shape)
out = [np.rot90(img, k=punchcard["rotation"][i]) for i, img in enumerate(out)]
print(out[0].shape, out[1].shape, out[2].shape)

In [ ]:
OVERLAY_HELPER.__generate_stack_rotation__(
    data_in=OLGen.data_in,
    punchcard=OLGen.data_punchcards["3_feature_rotation_(0, 0, 0, 3, 0, 2)"],
)

In [ ]:
OLGen.generate_overlay(patchsize=(512, 512))

In [ ]:
OLGen.data_in["feature_1"][0].dtype

In [ ]:
OLGen.__pad_input_imgs__(patchsize=(256, 256))
OLGen.data_in["feature_1"][0].dtype

In [ ]:
OLGen.__normalize_input_imgs__(max_val=255)
OLGen.data_in["feature_1"][0].dtype

In [ ]:
## Generate output
OLGen.mode = "overlay"
OLGen.generate_punchcards()
OLGen.generate_overlay(patchsize=(256, 256))

In [ ]:
## Data Inspector
files: list = pD.askFILES()
files = [np.load(f) for f in files]

fig, axs = plt.subplots(len(files), files[0].shape[0], figsize=(20, 10))
for j, file in enumerate(files):
    for i in range(files[0].shape[0]):
        axs[j, i].imshow(file[i])
        axs[j, i].axis("off")
        axs[j, i].set_title(f"File {j} - Patch {i} - Max: {np.max(file[i]):.2f}")